In [1]:
import litgen

options = litgen.LitgenOptions()
from litgen.demo import litgen_notebook_display

# Basic demonstration

## Export functions and their doc

In [2]:
code = """
// Subtracts two numbers: this will be the __doc__
int my_sub(int a, int b) { return a - b; }

// Standalone comment blocs 
// are also exported

int my_add(int a, int b) { return a + b; } // Adds two numbers (eol doc also works)

// div and mul: divide or multiply float numbers
// (This comment concerns the two grouped 
// functions below, and will be exported as such)
float div(float a, float b) { return a / b;} // Divide
float mul(float a, float b) { return a * b;} // Multiply

// This macro value will also be exported, as it matches the regex macro_define_include_by_name__regex
#define MY_VALUE 123

"""

options.macro_define_include_by_name__regex = "^MY_"

litgen_notebook_display.generate_and_display(options, code)

# Vectorize functions

In [5]:
code = """
float vec_mul(float a, float b) { return a * b; }
"""

options.fn_vectorize__regex = "^vec_"
options.fn_namespace_vectorize__regex = ".*"
litgen_notebook_display.generate_and_display(options, code)

# Return by reference

In [6]:
code = "MyWidget* MakeWidget();"

options.fn_return_force_policy_reference_for_pointers__regex = "^MakeWidget"
litgen_notebook_display.generate_and_display(options, code)

# Handling function parameters that are "modifiable for C++", but "immutable for python" 

float, int, str, etc. are immutable in python.

2 possibilities:
* Via `options.fn_params_replace_modifiable_immutable_by_boxed__regex`
* Via `options.fn_params_output_modifiable_immutable_to_return__regex`

In [7]:
code = """

// Those function will use a boxed value
void ToggleBoolPointer(bool *v)     { *v = !(*v); }
void ToggleBoolNull(bool *v = NULL) { if (v != NULL) *v = !(*v); }
void ToggleBoolReference(bool &v)   { v = !v; }

// this function's bnding will output a tuple [bool, str]
bool EditString(std::string* s)   { (*s) += "hello"; }
"""

options.fn_params_replace_modifiable_immutable_by_boxed__regex = r"^Toggle"
options.fn_params_output_modifiable_immutable_to_return__regex = r"^EditString$"
litgen_notebook_display.generate_and_display(options, code)

# Namespaces


In [37]:
code = """

int Blah();

namespace Foo
{
   int Blah();
   
   // This namespace is not exported
   namespace Details
   {
      int SecretSauce();
   }
}
"""

options.namespace_exclude__regex = r"[Ii]nternal|[Dd]etail"
litgen_notebook_display.generate_and_display(options, code)

# C style buffers to numpy arrays


## With fixed type

In [42]:
code = """
void SquareValues(float *values, int count);
"""
options.fn_params_replace_buffer_by_array__regex = r".*"
litgen_notebook_display.generate_and_display(options, code)

## With templated buffer types

In [4]:
code = """
template<typename AnyFloat> 
void DoubleValues(AnyFloat* values, int nb);
"""
options.fn_params_buffer_template_types = "^AnyFloat|T"
litgen_notebook_display.generate_and_display(options, code)

# (look at the generated pybind11 C++ code: it handles all numeric types)

While parsing a "function_decl", corresponding to this C++ code:
Position:2:1
    template<typename AnyFloat>
    ^
    void DoubleValues(AnyFloat* values, int nb);
While parsing a "function_decl", corresponding to this C++ code:
Position:2:1
    template<typename AnyFloat>
    ^
    void DoubleValues(AnyFloat* values, int nb);


# C Style string list
This function receives a C style string list (with 2 params). When translated to python, this will automatically be transformed to a `List[str]` 

In [4]:
code = "void display_messages(const char ** messages, int message_count);"
litgen_notebook_display.generate_and_display(options, code)

# Fixed size array parameters

In [55]:
code = """

// If the array is const, it can be cast to a python List[]
void foo_const(const int input[3]);

// If the array is modifiable, it will be boxed into several components
void foo_modif(int input[2]);
"""
litgen_notebook_display.generate_and_display(options, code)

# Classes, enums, structs

In [59]:
code = """
// This is the class doc. It will be published as MyClass.__doc__
class MyClass
{
public:
    MyClass(int factor = 10, const std::string& message = "hello"): factor(factor), message(message) {}
    ~MyClass() {}


    ///////////////////////////////////////////////////////////////////////////
    // Simple struct members
    ///////////////////////////////////////////////////////////////////////////
    int factor = 10, delta = 0;
    std::string message;


    ///////////////////////////////////////////////////////////////////////////
    // Stl container members
    ///////////////////////////////////////////////////////////////////////////

    // By default, modifications from python are not propagated to C++ for stl containers
    // (see https://pybind11.readthedocs.io/en/stable/advanced/cast/stl.html)
    std::vector<int> numbers;
    // However you can call dedicated modifying methods
    MY_API void append_number_from_cpp(int v) { numbers.push_back(v); }


    ///////////////////////////////////////////////////////////////////////////
    // Fixed size *numeric* array members
    //
    // They will be published as a py::array, and modifications will be propagated
    // on both sides transparently.
    ///////////////////////////////////////////////////////////////////////////

    int values[2] = {0, 1};
    bool flags[3] = {false, true, false};

    static const int const_static_value = 101;
    static int static_value;

    ///////////////////////////////////////////////////////////////////////////
    // Simple methods
    ///////////////////////////////////////////////////////////////////////////

    // calc: example of simple method
    MY_API int calc(int x) { return x * factor + delta; }
    // set_message: another example of simple method
    MY_API void set_message(const std::string & m) { message = m;}

    // unpublished_method: this function should not be published (no MY_API marker)
    int unpublished_method(int x) { return x * factor + delta + 3;}

    ///////////////////////////////////////////////////////////////////////////
    // Static method
    ///////////////////////////////////////////////////////////////////////////

    // Returns a static message
    MY_API static std::string static_message() { return std::string("Hi!"); }
};
"""
litgen_notebook_display.generate_and_display(options, code)